# Lab 4 (Part A): Regularized Linear Regression

In this Lab, you will implement regularized linear regression and use it to study models with different bias-variance properties. Make sure that you check the videos of lecture 5 before starting this Lab:
- Overfitting, Generalization, and Regularization: https://youtu.be/-zVe8upJMDU

In this part of the Lab, you will first implement regularized linear regression to predict the amount of water owing out of a dam using the change of water level in a reservoir. Then, you will examine the effects of bias vs. variance.

## Loading the data
We have a file `water-level-dataset.mat` which contains the dataset for our linear regression problem. The `.mat` file can be loaded in Python using `scipy.io.loadmat(..)` which returns a dictionary that contains 6 arrays. The follwing Python code load these arrays into the variables: `X`, `y`, `Xtest`, `ytest`, `Xval`, `yval`. So, our data is divided into three parts:
- A training set that your model will learn on: `X`, `y`
- A cross validation set for determining the regularization parameter: `Xval`, `yval`
- A test set for evaluating performance. These are *unseen* examples which your model did not see during training: `Xtest`, `ytest`.

Read the following code and print a small subset of these arrays to see what they look like.

In [1]:
%matplotlib notebook
from scipy.io import loadmat

mat = loadmat("datasets/water-level-dataset.mat")
# X and y correspond to a training set that your model will learn on.
X = mat["X"]
y = mat["y"].reshape(len(X))

# Xval and yval correspond to a cross validation set for determining the regularization parameter.
Xval = mat["Xval"]
yval = mat["yval"].reshape(len(Xval))

# Xtest and ytest correspond to a test set for evaluating performance. These 
# are unseen examples which your model will not see during training
Xtest = mat["Xtest"]
ytest = mat["ytest"].reshape(len(Xtest))


""" TODO:
You can print here a small subset of X, y, Xtest, ytest, Xval and Xval to see what they look like.
"""
print(X[:10], y[:10], Xtest[:10], ytest[:10], Xval[:10], yval[:10])


[[-15.93675813]
 [-29.15297922]
 [ 36.18954863]
 [ 37.49218733]
 [-48.05882945]
 [ -8.94145794]
 [ 15.30779289]
 [-34.70626581]
 [  1.38915437]
 [-44.38375985]] [ 2.13431051  1.17325668 34.35910918 36.83795516  2.80896507  2.12107248
 14.71026831  2.61418439  3.74017167  3.73169131] [[-33.31800399]
 [-37.91216403]
 [-51.20693795]
 [ -6.13259585]
 [ 21.26118327]
 [-40.31952949]
 [-14.54153167]
 [ 32.55976024]
 [ 13.39343255]
 [ 44.20988595]] [ 3.31688953  5.39768952  0.13042984  6.1925982  17.08848712  0.79950805
  2.82479183 28.62123334 17.04639081 55.38437334] [[-16.74653578]
 [-14.57747075]
 [ 34.51575866]
 [-47.01007574]
 [ 36.97511905]
 [-40.68611002]
 [ -4.47201098]
 [ 26.53363489]
 [-42.7976831 ]
 [ 25.37409938]] [ 4.17020201e+00  4.06726280e+00  3.18730676e+01  1.06236562e+01
  3.18360213e+01  4.95936972e+00  4.45159880e+00  2.22763185e+01
 -4.38738274e-05  2.05038016e+01]


## Visualizing the data
We will begin by visualizing the dataset containing historical records on the change in the water level, $x$, and the amount of water owing out of the dam, $y$. Note that the input data $X$ consists of only one feature corresponding to the "change in water level". Produce a scatter plot showing this feature vs. the output corresponding to the "water flowing out of the dam". Your plot should look like the following figure:
<img src="imgs/WaterDataScatterLab4A.png" width="400px" />


In [2]:
import matplotlib.pylab as plt

""" TODO:
Using the training dataset X and y, produce a scatter plot showing the "change in water level" on 
the x-axis and the "water flowing out of the dam" on the y-axis, as shown in the previous figure.
"""
fig, ax = plt.subplots()

ax.scatter(X, y, marker='x', color='r')
ax.set_xlabel("Change in water level (x)")
ax.set_ylabel("Water flowing out of the dam (y)")

fig.show()

<IPython.core.display.Javascript object>

## Adding a first column of ones to the dataset
As usual, before starting the implementation of our linear regression model, it might be helpful to use a modified version of our dataset which has an additional first column of ones.

In [3]:
import numpy as np

# This function takes a matrix as argument and returns a new matrix with an additional first column (of ones)
def add_all_ones_column(X):
    n, d = X.shape # dimension of the matrix X (n lines, d columns)
    XX = np.ones((n, d+1)) # new matrix of all ones with one additional column
    XX[:, 1:] = X # set X starting from column 1 (keep only column 0 unchanged)
    return XX

# The following line creates a new data matrix X_new with an additional first column (of ones)
X_new = add_all_ones_column(X) # for the training set
Xval_new = add_all_ones_column(Xval) # for the validation set

""" TODO:
You can print a small subset of X_new here to see how it looks like 
"""
# ...
print(X_new[:10])

[[  1.         -15.93675813]
 [  1.         -29.15297922]
 [  1.          36.18954863]
 [  1.          37.49218733]
 [  1.         -48.05882945]
 [  1.          -8.94145794]
 [  1.          15.30779289]
 [  1.         -34.70626581]
 [  1.           1.38915437]
 [  1.         -44.38375985]]


## Regularized linear regression cost function
Recall that regularized linear regression has the following cost function:
$$E(\theta) = \frac{1}{2n} \left [ \left ( \sum_{i=1}^{n} ( h_\theta(x^{(i)}) - y^{(i)} )^2 \right ) + \lambda \left ( \sum_{j=1}^{d} \theta_j^2 \right ) \right ],$$
where $\lambda$ is a regularization parameter which controls the degree of regularization (thus, help preventing overfitting). The regularization term puts a penalty on the overal cost function $E$. As the magnitudes of the model parameters $\theta_j$ increase, the penalty increases as well. Note that you should not regularize the $\theta_0$ term.

Complete the following code to write a function to calculate the regularized linear regression cost function. If possible, try to vectorize your code and avoid writing loops. When you are finished, call your cost function using $\theta$ initialized at `np.array([1, 1])` and $\lambda = 1$. You should then expect to see an output of about $303.993$.

In [4]:
""" TODO:
Write the definition of the regularized linear regression cost function. 
If possible, try to vectorize your code and avoid writing loops. The 
last argument lmd corresponds to the regularization term lambda.
"""
def E(theta, X, y, lmd):
    hyp = X @ theta
    squared_difference = (hyp - y) @ (hyp - y)
    reg_term = lmd*(theta[1:] @ theta[1:])
    return ((squared_difference) + (reg_term) ) / (2*len(y))


""" TODO:
Call your cost function with theta initialized to an array of ones, and 
lambda = 1. You should expect to see an output of about 303.993.
"""
theta = np.array([1, 1])
lmd = 1
print(E(theta, X_new, y, lmd))

303.9931922202643


## Regularized linear regression gradient
The partial derivative of regularized linear regression's cost for $\theta_j$ is defined as:
$$
\frac{\partial E(\theta)}{\partial \theta_0} = \frac{1}{n} \sum_{i=1}^{n} ( h_\theta(x^{(i)}) - y^{(i)} ) ~ x_j^{(i)}
\quad \quad \quad \quad \quad \quad \text{for } j = 0
$$

$$
\frac{\partial E(\theta)}{\partial \theta_j} = \left ( \frac{1}{n} \sum_{i=1}^{n} ( h_\theta(x^{(i)}) - y^{(i)} ) ~ x_j^{(i)} \right ) + \frac{\lambda}{n} \theta_j
\quad \quad \text{for } j \geq 1
$$

Complete the definition of the function `gradE(..)` in the code below, to calculate the gradient of the cost function $\nabla E(\theta)$. The function must return an array of the same length as $\theta$. When you are finished, call your gradient function using $\theta$ initialized at `np.array([1, 1])` and $\lambda = 1$. You should then expect to see a gradient of `[-15.30, 598.250]`.

In [5]:
""" TODO:
Write the definition of the gradient function. It should return an array containing 
the derivative of the cost function with respect to each parameter theta[j].
"""
def gradE(theta, X, y, lmd):
    theta_0 = ((((X @ theta) - y) @ X[:, 0]) / len(y))
    theta_j = ((((X @ theta) - y) @ X[:, 1:]) / len(y)) + (lmd*theta[1:]) / len(y)
    
    concatarray = np.append(theta_0, theta_j)
    return concatarray


""" TODO:
Call your gradient function with theta initialized to an array of ones, and 
lambda = 1. You should expect to see a gradient of approximatly [-15.30  598.250].
"""
# ...
theta = np.array([1, 1])
lmd = 1
print(gradE(theta, X_new, y, lmd))

[-15.30301567 598.25074417]


## Fitting linear regression
Once your cost function and gradient are working correctly, the following code will help you to compute the optimal values of $\theta$. We use `scipy.optimize.minimize(..)` to optimize the cost function.

In this part, we set the regularization parameter $\lambda$ to zero. Because our current implementation of linear regression is trying to fit a 2-dimensional $\theta$, regularization will not be incredibly helpful for a $\theta$ of such low dimension. In the later sections of this Lab, you will be using polynomial regression with regularization.

For more information about `scipy.optimize.minimize(..)`, you can visit:
- https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize
- https://docs.scipy.org/doc/scipy/reference/optimize.html

In [6]:
import scipy.optimize as op

theta = np.array([0, 0])  # Some initial parameters vector
lmda = 0 # We set lambda to zero this time.
print("Initial cost: ", E(theta, X_new, y, lmda))


""" TODO:
Use op.minimize(..) to minimize the cost function E. Then, print the 
optimal parameter vector theta and the final cost.
"""
#res = op.minimize(......)
res = op.minimize(E, theta, (X_new, y, lmda), 'TNC', gradE)
# Then, the optimal parameters can be accessed with: theta = res.x
theta = res.x
print(theta)
print("Final cost: ", E(theta, X_new, y, lmda))

Initial cost:  140.95412088055392
[13.0879035   0.36777923]
Final cost:  22.373906495108915


## Plotting the linear regression model
One you get the optimal parameters of $\theta$, you can call the function `plot_linear_fit(X, y, theta)` defined in the code below to plot the original dataset and the best fit line. Read the code carefully to see how such a line is ploted using the $\theta$ values. You will get a plot similar to the follwing figure.

The best fit line tells us that the model is not a good fit to the data because the data has a non-linear pattern. While visualizing the best fit as shown is one possible way to debug your learning algorithm, it is not always easy to visualize the data and model. In the next section, you will implement a function to generate learning curves that can help you debug your learning algorithm even if it is not easy to visualize the data.

<img src="imgs/RegLinePlotLab4A.png" width="400px" />

In [7]:
# This is a function that plots the original dataset (X, y) and the best fit line:
def plot_linear_fit(X, y, theta):
    fig, ax = plt.subplots()
    
    # Plottin the training data:
    ax.scatter(X[:, 0], y, marker="x", color="red")
    ax.set_xlabel("Change in water level (x)")
    ax.set_ylabel("Water flowing out of the dam (y)")
    
    # Plotting the line:
    x_min, x_max = np.min(X[:, 0]), np.max(X[:, 0])
    plot_x = np.arange(x_min, x_max) # range of values for the x axis
    plot_y = theta[0] + theta[1] * plot_x
    ax.plot(plot_x, plot_y, color="green", label="Best fit line")
    
    ax.set_title("Plot of the training data and best fit line")
    plt.legend()
    fig.show()


""" TODO:
Call the function plot_linear_fit(X, y, theta) with 
the optimal theta parameters that you got previously.
"""
plot_linear_fit(X, y, theta)

<IPython.core.display.Javascript object>

## Bias-variance
An important concept in machine learning is the bias-variance tradeoff. Models with high bias are not complex enough for the data and tend to underfit, while models with high variance overfit to the training data. In this part of the Lab, you will plot training and test errors on a learning curve to diagnose bias-variance problems.

You will now implement code to generate the learning curves that will be useful in debugging learning algorithms. Recall that a learning curve plots training and cross validation error as a function of training set size. Your job is to fill in the code below so that it returns a vector of errors for the training set and cross validation set.

To plot the learning curve, we need a training and cross validation set error for different training set sizes. To obtain different training set sizes, you should use different subsets of the original training set $X$. Specifically, for a training set size of $i$, you should use the first $i$ examples (i.e., `X[1:i]` and `y[1:i]`).

You can use the `scipy.optimize.minimize(..)` function to find the $\theta$ parameters. After learning the $\theta$ parameters, you should compute the error on the training and cross validation sets. Recall that the training error for a dataset is defined as:

$$E(\theta) = \frac{1}{2n} \sum_{i=1}^{n} ( h_\theta(x^{(i)}) - y^{(i)} )^2$$

In particular, note that the training error does not include the regularization term. One way to compute the training error is to use your existing cost function `E(..)` and set $\lambda$ to $0$ only when using it to compute the training error and cross validation error. When you are computing the training set error, make sure you compute it on the training subset (i.e., `X[0:i]` and `y[0:i]`) (instead of the entire training set). However, for the cross validation error, you should compute it over the entire cross validation set. You should store the computed errors in the vectors error train and error val.

When you are finished, print the learning curves and produce a plot similar to the following figure. From the figure, you can observe that both the train error and cross validation error are quite high even when the number of training examples is increased. This reflects a **high bias** problem in the model - the linear regression model is too simple and is unable to fit our dataset well. In the next section, you will implement polynomial regression to fit a better model for this dataset.

<img src="imgs/LearningCurvLab4A.png" width="400px" />

In [8]:
# This function takes some training data, an initial theta and a regularization term 
# lmd, and returns the optimal prameters vector theta of a linear regression. It uses 
# scipy.optimize.minimize(..), the cost function E(..) and the gradient gradE(..)
def trainLinearReg(X, y, theta_init, lmd):
    res = op.minimize(E, theta_init, (X, y, lmd), 'TNC', gradE)
    return res.x # the best/final parameters vector theta


""" TODO:
Inside a loop which iterates over the range of n (number of training examples), find the optimal theta by 
calling trainLinearReg(..) using the first i examples (X_new[:i], y[:i]), then compute the corresponding 
training error err_train (using X_new[:i], y[:i]) and the validation error err_val (using Xval_new, yval) ...
"""
errs_train, errs_val = [], [] # Lists to save the training and validation errors
theta_init = np.array([0, 0])
lmd = 0
for i in range(2, len(y)): # Start from 2 examples at least
    #pass
    # TODO: find the optimal theta parameters using the i first training examples
    op_theta = trainLinearReg(X_new[:i], y[:i], theta_init, lmd)
    
    # TODO: compute the training error using the optimal theta and the i first training examples
    err_train = E(op_theta, X_new[:i], y[:i], lmd) #sum(((X_new[:i] @ op_theta - y[:i]))**2) / (2*len(y[:i]))
    
    # TODO: compute the validation error using the optimal theta and the validation dataset
    err_val = E(op_theta, Xval_new, yval, lmd) #sum(((Xval_new[:i] @ op_theta - yval[:i]))**2) / (2*len(yval[:i]))
    
    errs_train.append(err_train)
    errs_val.append(err_val)


""" TODO:
Complete the code below to plot the learning curve
"""
fig, ax = plt.subplots()
print()
# TODO: plot the "number of training examples" vs the training errors
x = np.arange(2, len(y))
ax.plot(x, errs_train, color='b', label='Training error')

# TODO: plot the "number of training examples" vs the validation errors
ax.plot(x, errs_val, color='orange', label='Validation error')

ax.set_xlabel("Number of training examples")
ax.set_ylabel("Error")
ax.set_title("Linear regression learning curve")
ax.legend()
fig.show()

<IPython.core.display.Javascript object>

## Polynomial regression
The problem with our linear model was that it was too simple for the data and resulted in underfitting (high bias). In this part of the exercise, you will address this problem by adding more features. For using polynomial regression, our hypothesis has the form:
$$
\begin{align}
h_\theta(x) &= \theta_0 + \theta_1 * (\text{waterLevel}) + \theta_2 * (\text{waterLevel})^2 + \dots + \theta_p * (\text{waterLevel})^p \\
&= \theta_0 + \theta_1 x_1 + \theta_2 x_2 + \dots + \theta_p x_p
\end{align}
$$
Notice that by defining $x_1 = (\text{waterLevel}), x_2 = (\text{waterLevel})^2, \dots, x_p = (\text{waterLevel})^p$, we obtain a linear regression model where the features are the various powers of the original value (waterLevel). Note that you don't have to account for the zero'th power in this function.

Now, you will add more features using the higher powers of the existing feature $x$ in the dataset. Your task in this part is to complete the following code so that the function `polyFeatures(..)` maps the original training set $X$ of size $n \times 1$ into its higher powers. Specifically, when a training set $X$ of size $n \times 1$ is passed into the function, the function should return a $n \times p$ matrix `X_poly`.

In [9]:
""" TODO:
Write the definition of the function polyFeatures(X, p) which takes 
as arguement the original training set X and the degree of the polynomial p. 
The function should return a new set X_poly of p features (i.e. matrix of len(X) rows and p columns).
"""
def polyFeatures(X, p):
    water_feature = X
   # print("X = ", X)
    for i in range(2, p + 1):
        new_array = np.power(water_feature, i)
        #print("new = ", new_array)
        X = np.append(X, new_array, axis = 1)    
    return X 

""" TODO:
You can call your polyFeatures(...) function here on some exampls to check if it works correctly.
"""
test = np.array([[2], 
                 [3]])
#print("X = ", X)
#print()
polyFeatures(test, 8) 

array([[   2,    4,    8,   16,   32,   64,  128,  256],
       [   3,    9,   27,   81,  243,  729, 2187, 6561]])

After you have completed the function `polyFeatures`, we will proceed to train polynomial regression using your linear regression cost function. Keep in mind that even though we have polynomial terms in our feature vector, we are still solving a linear regression optimization problem. The polynomial terms have simply turned into features that we can use for linear regression. We are using the same cost function and gradient that you wrote for the earlier part of this Lab.

In the Python code below, we are using a polynomial of degree 8. The function `polyFeatures(..)` is therefore applied with $p=8$ to the training set `X`, the validation set `Xval` and the test set `Xtest`, to get the corresponding projected data `X_poly`, `Xval_poly` and `Xtest_poly`.

It turns out that if we run the training directly on the projected data `X_poly`, it will not work well as the features would be badly scaled (e.g., an example with $x_1 = 40$ will have a feature $x_8 = 40^8$). Therefore, we need to use feature normalization. In the following Python code, we first normalize the features of the training set and add to it an additional first column of ones to get `X_poly_normalized`. The parameters used for normalization are stored in variables `mu` and `sigma`. Then, we train the linear regression model (i.e. find the optimal parameters vector $\theta$) by calling the function `trainLinearReg(..)` using our new training set `X_poly_normalized`, `y`, an initial $\theta$ vector of dimension $p+1$ (as we have $p$ features now), and a value of $\lambda$ set to $0$ for now (no regularization). Then, we plot two figures which looks like follows. From the first figure, you should see that the polynomial fit is able to follow the data-points very well - thus, obtaining a low training error. However, the polynomial fit is very complex and even drops off at the extremes. This is an indicator that the polynomial regression model is overfitting the training data and will not generalize well. To better understand the problems with the unregularized ($\lambda = 0$) model, you can see from the second figure that the learning curve shows the same effect where the training error is low, but the validation error is high. There is a gap between the training and cross validation errors, indicating a high variance problem.

We have already implemented all this for you. You just need to read the code and then run it.

<img src="imgs/Lab4APolyPlotLmd0.png" width="600px" />

In [10]:
# The regularization parameter lmd
lmd = 0

# Using polyFeatures(..) with p=8 to map the original data to a higher dimension
p = 8
X_poly = polyFeatures(X, p) # on the training set
Xval_poly = polyFeatures(Xval, p) # on the validation set
Xtest_poly = polyFeatures(Xtest, p) # on the test set

# mean vector and standard deviation vector
mu = np.mean(X_poly, axis=0)
sigma = np.std(X_poly, axis=0)

# normalizing the training set and validation set using mu and sigma and adding an aditional first column of ones
X_poly_normalized = add_all_ones_column((X_poly - mu) / sigma)
Xval_poly_normalized = add_all_ones_column((Xval_poly - mu) / sigma)

# Training to find the optimal parameters vector theta
theta_init = np.zeros(p+1) # we p features, so we need p+1 parameters (as we have theta_0)
theta = trainLinearReg(X_poly_normalized, y, theta_init, lmd)

# Plotting the dataset and the polynomial regression curve
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.set_xlabel("Change in water level (x)")
ax1.set_ylabel("Water flowing out of the dam (y)")
ax1.set_title(r"Polynomial Regression Fit ($\lambda = {}$)".format(lmd))

ax2.set_xlabel("Number of training examples")
ax2.set_ylabel("Error")
ax2.set_title(r"Polynomial Regression Learning Curve ($\lambda = {}$)".format(lmd))

x_plot = np.linspace(-60, 40, 100)
x_plot_poly = polyFeatures(x_plot.reshape(len(x_plot), 1), p)
x_plot_poly_normalized = add_all_ones_column((x_plot_poly - mu) / sigma)
y_plot = x_plot_poly_normalized @ theta
ax1.plot(x_plot, y_plot, linestyle="--", color="green", label="Polynomial fit")
ax1.scatter(X[:, 0], y, marker="x", color="red", label = "Training examples")

# Plotting the learning curves using the training and validation sets
errs_train, errs_val = [], []
for i in range(2, len(y)):
    theta = trainLinearReg(X_poly_normalized[:i], y[:i], theta_init, lmd)
    errs_train.append( E(theta, X_poly_normalized[:i], y[:i], 0) )
    errs_val.append( E(theta, Xval_poly_normalized, yval, 0) )
    
ax2.plot(range(2, len(y)), errs_train, label="Training error")
ax2.plot(range(2, len(y)), errs_val, label="Validation error")

ax1.legend()
ax2.legend()
plt.show()

<IPython.core.display.Javascript object>

## Adjusting the regularization parameter $\lambda$
In this section, you will get to observe how the regularization parameter affects the bias-variance of regularized polynomial regression. You should now modify the $\lambda$ parameter (i.e. variable `lmd`) in previous code and try $\lambda = 1$ and $\lambda = 50$ (you could also try with other values of $\lambda$). For each of these values, the above code should generate two figures (the polynomial fit to the data and the learning curve).

For $\lambda = 1$, you should see a polynomial fit that follows the data trend well and a learning curve showing that both the validation and training error converge to a relatively low value. This shows the $\lambda = 1$ regularized polynomial regression model does not have the high-bias or high-variance problems. In fact, it achieves a good trade-off between bias and variance. For $\lambda = 50$, you should see a polynomial fit that does not follow the data well. In this case, there is too much regularization and the model is unable to fit the training data.

## Selecting $\lambda$ using the validation set
From the previous section, you observed that the value of $\lambda$ can significantly affect the results of regularized polynomial regression on the training and cross validation set. In particular, a model without regularization ($\lambda = 0$) fits the training set well, but does not generalize. Conversely, a model with too much regularization (e.g. $\lambda = 50$) does not fit the data well. A good choice of $\lambda$ (e.g., $\lambda = 1$ for this dataset) can provide a good fit to the data.

In this section, you will implement an automated method to select the $\lambda$ parameter. Concretely, you will use a cross validation set to evaluate how good each $\lambda$ value is. After selecting the best $\lambda$ value using the validation set, we can then evaluate the model on the test set to estimate how well the model will perform on actual unseen data.

Your task is to complete the code below. Specifically, you should use the `trainLinearReg(..)` function to train the model using different values of $\lambda$ and compute the training error and validation error. You should try $\lambda$ in the following range: `[0, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10]`. You should also plot a curve that shows the validation error and training error with respect to the different values of $\lambda$. You should see a plot similar to the following figure, which allows you to select which $\lambda$ parameter to use. In this figure, we can see that the best value of $\lambda$ is around $3$ (as it gives the smallest validation error).

<img src="imgs/Lab4AerrVSlambda.png" width="400px" />

*Note:* Due to randomness in the training and validation splits of the dataset, the cross validation error can sometimes be lower than the training error.

In [11]:
lmd_range = [0, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10]
errs_train, errs_val = [], []

""" TODO:
For each value of lmd you should call trainLinearReg(..) to train a model using the training set 
X_poly_normalized, y, then compute the training error (on the training set: X_poly_normalized, y) 
and the validation error (on the validation set: Xval_poly_normalized, yval).
"""
for lmd in lmd_range:
    op_theta = trainLinearReg(X_poly_normalized, y, theta_init, lmd) 
    
    err_train = E(op_theta, X_poly_normalized, y, lmd)
    
    err_val = E(op_theta, Xval_poly_normalized, yval, lmd)
    
    errs_train.append(err_train)
    errs_val.append(err_val)

""" TODO:
Plot a curve that shows the validation error and training error with respect to lambda
"""
fig, ax = plt.subplots()

# Plot of the training errors vs lambda values
ax.plot(lmd_range, errs_train, color='b', label='Training error')
# Plot of the validation errors vs lambda values
ax.plot(lmd_range, errs_val, color='orange', label='Validation error')

ax.set_xlabel("lambda")
ax.set_ylabel("Error")
ax.legend()
fig.show()


<IPython.core.display.Javascript object>

## Computing test set error
In the previous section, you implemented code to compute the validation error for various values of the regularization parameter $\theta$. However, to get a better indication of the model's performance in the real world, it is important to evaluate the "*final*" model on a test set that was not used in any part of training (that is, it was neither used to select the regularization parameter $\lambda$, nor to learn the model parameters $\theta$).

In the code below, you should compute the test error using the best value of $\theta$ you found (using $\lambda = 3$).

In [24]:
""" TODO:
Find the best parameters vector theta by training the polynomial regression 
model using lambda = 3. Then compute the error on the test set.
Note: don't forget to normalize the test set before.
"""
lmd = 3
theta = trainLinearReg(X_poly_normalized, y, theta_init, lmd)
Xtest_poly_norm = (Xtest_poly - mu) / sigma #np.std(Xtest_poly, axis=0)
Xtest_poly_norm_new = add_all_ones_column(Xtest_poly_norm)
#print(Xtest_poly_norm_new)
#print(Xtest_poly)
err_test = E(theta, Xtest_poly_norm_new, ytest, lmd)
print("err_test =", err_test)

err_test = 9.522061477564405


## (OPTIONAL) Performing a 10-fold-cross-validation

This section is optional.

Combine the training and validation sets into one set `Xall`, `yall`. Note: in Python you can combine two datasets A and B (two numpy arrays) into one dataset using `AB = np.append(A, B, axis=0)`.

Implement 10-fold-cross-validation to evaluate to find the best value for the hyperparameter $\lambda$. This will be the value that gives you the best generalization error.

Finally, use this best value of $\lambda$ to train a model on the whole dataset `Xall`, `yall`, and then test this model on the test dataset `Xtest`, `ytest` and compute the test error.

In [23]:
""" TODO
Plot the learning curves based on randomly selected examples as described above.
"""

' TODO\nPlot the learning curves based on randomly selected examples as described above.\n'